In [1]:
import pandas as pd
from pathlib import Path
from astropy.io import fits

# ==============================
# Inputs
# ==============================
SPW_ORDER = [15, 16, 17, 2, 3, 4, 5, 6, 8]
DATASETS  = ["23", "25", "32", "36"]

# Example path pattern you gave:
# ../paper1/spw/23_pb/spw2-2.5arcsec-nit5000-awproject.pbcor.fits

base_dir = Path("../paper1/spw")
suffix   = "-2.5arcsec-nit5000-awproject.pbcor.fits"

out_csv = Path("../paper1/spw_beam_keywords_BMAJ_BMIN_BPA.csv")

# ==============================
# Helper
# ==============================
def read_beam_keywords(fits_path: Path):
    """
    Returns (BMAJ, BMIN, BPA, comments_dict) from primary header.
    Values are typically:
      - BMAJ, BMIN in degrees
      - BPA in degrees (position angle)
    """
    with fits.open(fits_path) as hdul:
        hdr = hdul[0].header

    bmaj = hdr.get("BMAJ", None)
    bmin = hdr.get("BMIN", None)
    bpa  = hdr.get("BPA",  None)

    # Optional: header comments (units often appear in comments)
    comments = {
        "BMAJ_comment": hdr.comments["BMAJ"] if "BMAJ" in hdr else None,
        "BMIN_comment": hdr.comments["BMIN"] if "BMIN" in hdr else None,
        "BPA_comment":  hdr.comments["BPA"]  if "BPA"  in hdr else None,
    }

    return bmaj, bmin, bpa, comments

# ==============================
# Main loop
# ==============================
rows = []

for ds in DATASETS:
    ds_dir = base_dir / f"{ds}_pb"

    for spw in SPW_ORDER:
        fits_path = ds_dir / f"spw{spw}{suffix}"

        if not fits_path.exists():
            print(f"[MISSING] {fits_path}")
            rows.append({
                "dataset": ds,
                "spw": spw,
                "file": str(fits_path),
                "exists": False,
                "BMAJ": None,
                "BMIN": None,
                "BPA": None,
                "BMAJ_comment": None,
                "BMIN_comment": None,
                "BPA_comment": None,
            })
            continue

        try:
            bmaj, bmin, bpa, comments = read_beam_keywords(fits_path)

            print(f"[OK] dataset={ds} spw={spw}  BMAJ={bmaj}  BMIN={bmin}  BPA={bpa}")

            rows.append({
                "dataset": ds,
                "spw": spw,
                "file": str(fits_path),
                "exists": True,
                "BMAJ": bmaj,
                "BMIN": bmin,
                "BPA": bpa,
                **comments
            })

        except Exception as e:
            print(f"[ERROR] {fits_path}  -> {e}")
            rows.append({
                "dataset": ds,
                "spw": spw,
                "file": str(fits_path),
                "exists": True,
                "BMAJ": None,
                "BMIN": None,
                "BPA": None,
                "BMAJ_comment": None,
                "BMIN_comment": None,
                "BPA_comment": None,
                "error": str(e),
            })

# ==============================
# Save CSV
# ==============================
df = pd.DataFrame(rows)
df = df.sort_values(["dataset", "spw"]).reset_index(drop=True)

out_csv.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out_csv, index=False)

print(f"\nSaved: {out_csv}")
print(df.head(10))


[OK] dataset=23 spw=15  BMAJ=0.01694444444444  BMIN=0.01694444444444  BPA=0.0
[OK] dataset=23 spw=16  BMAJ=0.01694444444444  BMIN=0.01694444444444  BPA=0.0
[OK] dataset=23 spw=17  BMAJ=0.01694444444444  BMIN=0.01694444444444  BPA=0.0
[OK] dataset=23 spw=2  BMAJ=0.01694444444444  BMIN=0.01694444444444  BPA=0.0
[OK] dataset=23 spw=3  BMAJ=0.01694444444444  BMIN=0.01694444444444  BPA=0.0
[OK] dataset=23 spw=4  BMAJ=0.01694444444444  BMIN=0.01694444444444  BPA=0.0
[OK] dataset=23 spw=5  BMAJ=0.01694444444444  BMIN=0.01694444444444  BPA=0.0
[OK] dataset=23 spw=6  BMAJ=0.01694444444444  BMIN=0.01694444444444  BPA=0.0
[OK] dataset=23 spw=8  BMAJ=0.01694444444444  BMIN=0.01694444444444  BPA=0.0
[OK] dataset=25 spw=15  BMAJ=0.01694444444444  BMIN=0.01694444444444  BPA=0.0
[OK] dataset=25 spw=16  BMAJ=0.01694444444444  BMIN=0.01694444444444  BPA=0.0
[OK] dataset=25 spw=17  BMAJ=0.01694444444444  BMIN=0.01694444444444  BPA=0.0
[OK] dataset=25 spw=2  BMAJ=0.01694444444444  BMIN=0.01694444444444  B